## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [17]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [18]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002394BB1BBF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002394BBF8080>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [19]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you! \n\nBeing a Chief AI Engineer is a fascinating role. What are you currently working on that you're most excited about?  \n\nI'm eager to learn more about your work in the field of AI.\n", response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 27, 'total_tokens': 85, 'completion_time': 0.119772602, 'prompt_time': 0.005220794, 'queue_time': None, 'total_time': 0.124993396}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd15e7b9-cae2-405e-b95d-734f5dc1cf44-0', usage_metadata={'input_tokens': 27, 'output_tokens': 58, 'total_tokens': 85})

In [20]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer!  \n\nI remember that from our first conversation.  😄  \n\nIs there anything specific you'd like to talk about regarding your work as a Chief AI Engineer?  Perhaps a project you're particularly excited about? \n\n", response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 102, 'total_tokens': 164, 'completion_time': 0.129012892, 'prompt_time': 0.017027962, 'queue_time': None, 'total_time': 0.146040854}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a8058007-be0a-45eb-bc56-5a5e00e8d070-0', usage_metadata={'input_tokens': 102, 'output_tokens': 62, 'total_tokens': 164})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [21]:
!pip install langchain_community

In [22]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [23]:
config={"configurable":{"session_id":"chat1"}}

In [24]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [25]:
response.content

"Hi Krish, it's nice to meet you! \n\nThat's a very interesting role. What kind of AI projects are you working on? \n\nI'm always eager to learn about the latest developments in the field.\n"

In [26]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Your name is Krish.  I remember from our earlier conversation! 😊 \n\nIs there anything else I can help you with today?  \n', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 92, 'total_tokens': 123, 'completion_time': 0.063727092, 'prompt_time': 0.00881226, 'queue_time': None, 'total_time': 0.072539352}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2e2603dd-5ebc-4c73-b02e-4ca6bab07bc0-0', usage_metadata={'input_tokens': 92, 'output_tokens': 31, 'total_tokens': 123})

In [27]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I have no memory of past conversations or any information about you, including your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [28]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hi John! It's nice to meet you.  \n\nWhat can I do for you today? 😄  \n"

In [29]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is John.  \n\nIs there anything else I can help you with, John? 😊 \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [30]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [31]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]})

AIMessage(content="Hi Krish, it's nice to meet you! 👋\n\nHow can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 36, 'total_tokens': 62, 'completion_time': 0.053036343, 'prompt_time': 0.002612557, 'queue_time': None, 'total_time': 0.0556489}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8804726-c972-4292-a892-025c6dcc090f-0', usage_metadata={'input_tokens': 36, 'output_tokens': 26, 'total_tokens': 62})

In [32]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [33]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hi Krish, it's nice to meet you!  \n\nWhat can I do for you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 36, 'total_tokens': 63, 'completion_time': 0.054679621, 'prompt_time': 0.004616875, 'queue_time': None, 'total_time': 0.059296496}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-08215788-87f9-4c5d-9786-638700f0ec0a-0', usage_metadata={'input_tokens': 36, 'output_tokens': 27, 'total_tokens': 63})

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Krish!  I remember.  \n\nIs there anything else I can help you with?\n'

In [35]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [36]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण!  😊 \n\nआपका नाम सुन्दर है। \n\nमुझे आपसे बात करने में खुशी हो रही है। क्या मैं आपकी कोई मदद कर सकता हूँ? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [37]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [38]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'नमस्ते, कृष्ण!  😊 \n\nआप कैसे हैं?  \n'

In [39]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [40]:
response.content

'आपका नाम कृष्ण है। 😄 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [41]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

C:\Users\ACER\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ACER\OneDrive\Desktop\Python\annclassification\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ACER\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to 

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [42]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor? 😋🍦\n"

In [43]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2".  \n\n\n\nLet me know if you\'d like to try another one! 😊 \n'

In [44]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [45]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n"

In [46]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I have no memory of past conversations.  If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"